In [34]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
import fastf1 as f1
import plotly.graph_objects as go

In [35]:
f1.Cache.enable_cache("C:\\Users\\mailn\\Downloads\\cache")

In [36]:
def plot_car_comp(trk, sector_length):
    track=trk
    minisector_length=sector_length
    session_21 = f1.get_session(2021, track, 'Q')
    session_22 = f1.get_session(2022, track, 'Q')
    session_21.load()
    session_22.load()
    vers_21 = session_21.laps.pick_driver('VER').pick_fastest().get_telemetry()
    vers_22 = session_22.laps.pick_driver('VER').pick_fastest().get_telemetry()
    vers_21 = vers_21.reset_index().drop(columns=['index']).copy()
    vers_22 = vers_22.reset_index().drop(columns=['index']).copy()
    vers_21['season'] = ['2021' for _ in range(vers_21.shape[0])]
    vers_22['season'] = ['2022' for _ in range(vers_22.shape[0])]
    vers_21['mini_sector'] = pd.Series(list((vers_21['Distance']//minisector_length)+1)).astype('int')
    vers_22['mini_sector'] = pd.Series(list((vers_22['Distance']//minisector_length)+1)).astype('int')
    vers_21['shift_dist']=vers_21['Distance'].shift(+1).fillna(0.0)
    vers_21['cover_dist'] = vers_21['Distance'] - vers_21['shift_dist']
    vers_21['mod_time'] = pd.to_numeric(vers_21['Time'])/1000000000
    vers_21['shift_time'] =  vers_21['mod_time'].shift(+1).fillna(0.000)
    vers_21['cover_time'] = vers_21['mod_time'] - vers_21['shift_time']
    vers_21_proc = vers_21.groupby('mini_sector')[['cover_dist', 'cover_time']].sum()
    dist21 = vers_21_proc['cover_dist']*18
    time21 = vers_21_proc['cover_time']*5
    avg_speed21 = dist21/time21
    vers_21_proc['minisector_speed_2021'] = avg_speed21
    vers_22['mini_sector'] = pd.Series(list((vers_22['Distance']//minisector_length)+1)).astype('int')
    vers_22['mini_sector'] = pd.Series(list((vers_22['Distance']//minisector_length)+1)).astype('int')
    vers_22['shift_dist']=vers_22['Distance'].shift(+1).fillna(0.0)
    vers_22['cover_dist'] = vers_22['Distance'] - vers_22['shift_dist']
    vers_22['mod_time'] = pd.to_numeric(vers_22['Time'])/1000000000
    vers_22['shift_time'] =  vers_22['mod_time'].shift(+1).fillna(0.000)
    vers_22['cover_time'] = vers_22['mod_time'] - vers_22['shift_time']
    vers_22_proc = vers_22.groupby('mini_sector')[['cover_dist', 'cover_time']].sum()
    dist22 = vers_22_proc['cover_dist']*18
    time22 = vers_22_proc['cover_time']*5
    avg_speed22 = dist22/time22
    vers_22_proc['minisector_speed_2022'] = avg_speed22
    vers_speed_comp = vers_21_proc.copy()
    vers_speed_comp['minisector_speed_2022']=vers_22_proc['minisector_speed_2022']
    fast_sectors_2022 = list((vers_speed_comp['minisector_speed_2022'] > vers_speed_comp['minisector_speed_2021'])[(vers_speed_comp['minisector_speed_2022'] > vers_speed_comp['minisector_speed_2021'])].index)
    fast_sectors_2021 = list((vers_speed_comp['minisector_speed_2021'] > vers_speed_comp['minisector_speed_2022'])[(vers_speed_comp['minisector_speed_2021'] > vers_speed_comp['minisector_speed_2022'])].index)
    vers_22_plot = vers_22[vers_22['mini_sector'].isin(fast_sectors_2022)]
    vers_21_plot = vers_21[vers_21['mini_sector'].isin(fast_sectors_2021)]
    print(f"Lap time in 2021: {vers_21.iloc[-1, :]['mod_time']} seconds")
    print(f"Lap time in 2022: {vers_22.iloc[-1, :]['mod_time']} seconds")
    fig = go.Figure()
    fig.update_layout(template="plotly_dark", title="Comparing the speeds of 2021 & 2022 RedBull F1 cars of Max Verstappen in the Qualifiers.<br>Austin, Texas circuit(COTA) showing the faster Redbull car in the different mini sectors.", font_size=12, font=dict(family='Overpass'))
    fig.update_xaxes(showgrid=False, visible=False)
    fig.update_yaxes(showgrid=False, visible=False)
    fig.add_trace(go.Scatter(x=vers_21_plot["X"], y=vers_21_plot["Y"], name="2021 RedBull car", mode='markers', marker=dict(color='royalblue', size=10)))
    fig.add_trace(go.Scatter(x=vers_22_plot["X"], y=vers_22_plot["Y"], name="2022 Redbull car", mode='markers', marker=dict(color='tomato', size=10)))
    fig.update_layout(shapes=[go.layout.Shape(
        type='rect',
        xref='paper',
        yref='paper',
        x0=0,
        y0=-0.1,
        x1=1.01,
        y1=1.02,
        line={'width': 3, 'color': 'gray'})
                             ]
                     )
    fig.show()
    return vers_speed_comp

In [37]:
df1 = plot_car_comp('Austin', 50)

core           INFO 	Loading data for United States Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['33', '44', '11', '77', '16', '55', '3', '4', '10', '22', '31', '5', '99', '14', '63', '18', '6', '7', '47', '9']
core           INFO 	Loading data for United States Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cach

Lap time in 2021: 92.91 seconds
Lap time in 2022: 94.448 seconds
